In [1]:
using Pipe
using DataFrames
# using StatsPlots
using CSV
# using RollingFunctions
using Query
# using GLMakie

using JSON
using DataFrames
using Dates
using Colors
# using ColorSchemes
using Indicators

import Colors
using Plots
using Interact
# using JuliaDB
using Interact
# using Wavelets,WaveletsExt
# using LinearAlgebra
# import Pkg; Pkg.add("ShiftedArrays")
# using ShiftedArrays
include("utils.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-6720092924660885695\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-6314971597406919308\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-6720092924660885695\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

calc_wavelet (generic function with 1 method)

In [2]:
json_data_folder = "../user_data/data/binance"
json_filename = "BTC_USDT-1m.json"
json_filepath = "$json_data_folder/$json_filename"
f = open(json_filepath, "r")
json_data = JSON.parse(f)
close(f)

In [3]:
dfa = DataFrame(unix_time = Int64[], open=Float64[], high=Float64[], low=Float64[], close=Float64[], volume=Float64[])
for r in json_data push!(dfa, r) end
dfa.unix_time .= dfa.unix_time ./1000
# dfa.time .= unix2datetime.(dfa.unix_time)

insertcols!(dfa, :time => unix2datetime.(dfa.unix_time))
nrow(dfa)

129177

In [13]:
start_date = DateTime(2021, 4, 1)
end_date = DateTime(2021, 5, 27)
vdf = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]
ut, tt, hh, ll, cc = vdf.unix_time, vdf.time, vdf.high, vdf.low, vdf.close
# h14max = [fill(vdf[1,:high],8-1)...,rollmax(vdf.high, 8)...]
#  length(h14max), length(vdf.close)
# hlc = [hh ll cc]
length(tt)

80207

In [14]:
ar = cc
# ema = Indicators.ema
# ema26 = ema(ar,n=26)
# ema50 = ema(ar,n=330)
# ema100 = ema(ar,n=100)
# ema200 = ema(ar,n=200)
# ema400 = ema(ar,n=400)
# sma = Indicators.sma
# sma26 = sma(ar,n=26)
# sma50 = sma(ar,n=50)
# sma100 = sma(ar,n=100)
# sma200 = sma(ar,n=200)
# sma400 = sma(ar,n=400)
# sma800 = sma(ar,n=800)
1
ar_emas = Dict([i => ema(ar, n=2^i) for i in 4:9])

Dict{Int64, Vector{Float64}} with 6 entries:
  5 => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  39134.4, 39140.4, …
  4 => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  39214.1, 39216.5, …
  6 => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  39044.3, 39050.2, …
  7 => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  38951.3, 38955.7, …
  9 => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  38866.7, 38868.2, …
  8 => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  38853.5, 38856.4, …

In [ ]:
dfp = DataFrame(unix_time = Int64[], open=Float64[], high=Float64[], low=Float64[], close=Float64[], volume=Float64[])


In [25]:
ar = cc
ar_len = length(ar)
plot_box = Observable{Any}(dom"div"())


window_size = 60 * 8

num_windows = Int64(trunc(ar_len/window_size*4))

wnd_slider = slider(1:num_windows; label="start", default=7)

# wnd_slider = window_slider(length(ar))
# rng = Observable{AbstractRange{Int64}}(1:1)
# map!(wnd -> wnd[:window_start]:(wnd[:window_start]+wnd[:window_size]-1), rng, wnd)

# plt = Interact.@map plot(ar[&wnd[:window_start]:(&wnd[:window_start]+&wnd[:window_size]-1)])

# window_start = Interact.@map &wnd[:window_start]
# window_size = Interact.@map &wnd[:window_size]


function draw_it_all(window_no)
    window_start = Int(window_no * window_size / 4)
    window_end = min(ar_len,window_start+window_size-1)
    rng = window_start:window_end
    plt = plot(ar[rng], legend=nothing, size=(1200,300)) 
    
#     ema_6 = ar_emas[6]
#     ema_7 = ar_emas[6]
    
    plt = plot!(ar_emas[6][rng])
    plt = plot!(ar_emas[7][rng])

    dom"div"(plt)
end

map!(draw_it_all, plot_box, wnd_slider)
ui = dom"div"(plot_box,wnd_slider)


(div
  Observable{Any} with 0 listeners. Value:
Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())
  Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["start"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 668, :default => 7, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :min => 1, :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Int64} with 2 listeners. Value:
334, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000011837a980, Task (runnable) @0x000000011837a980), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\

In [17]:
num_windows, window_size


(55, 1440)

In [12]:
60 * 24 * 7

10080

In [ ]:
maema_1_ar

In [ ]:
wnd[:window_size] 

In [ ]:
wave_slider[]

In [ ]:
i = 1000
i - max(1,i - 512)

In [ ]:
wt = wavelet(WT.Coiflet{4}(), WT.Filter, WT.Periodic)

In [ ]:
2^8

In [ ]:
length(ar)